In [2]:
import pandas as pd
import glob
import os
from datetime import datetime

# --- 1. DOSYALARI BULMA ---

dosyalar = glob.glob("*.csv")

# --- 2. AKILLI SIRALAMA (Kronolojik) ---

def tarih_ayikla(dosya_ismi):
    
    parcalar = dosya_ismi.split('_')
    yil = parcalar[0]
    ay_ismi = parcalar[1]
    
    tarih_objesi = datetime.strptime(f"{yil}-{ay_ismi}", "%Y-%B")
    return tarih_objesi

# Listeyi tarihe göre sırala
try:
    dosyalar.sort(key=tarih_ayikla)
    print("Dosyalar tarih sırasına (Ocak -> Aralık) göre dizildi.\n")
except Exception as e:
    print("Sıralama uyarısı: Dosya isimlendirmesi standart dışı olabilir, alfabetik devam ediliyor.")
    dosyalar.sort() 

print(f"📂 Bulunan Dosyalar ({len(dosyalar)} Adet):")
print(dosyalar) 


# --- 3. SÜTUN KONTROLÜ (Değişmedi) ---
sutun_kontrol = {}

for dosya in dosyalar:
    try:
        # Sadece başlıkları oku
        df_temp = pd.read_csv(dosya, nrows=0)
        columns = tuple(df_temp.columns)
        
        if columns not in sutun_kontrol:
            sutun_kontrol[columns] = []
        sutun_kontrol[columns].append(dosya)
        
    except Exception as e:
        print(f"Hata: {dosya} okunurken sorun oluştu: {e}")

# --- 4. RAPOR ---
print("SÜTUN KONTROL RAPORU")
if len(sutun_kontrol) == 1:
    print("MÜKEMMEL! Tüm dosyaların sütun isimleri ve sırası birebir aynı.")
    print(f"Sütunlar: {list(sutun_kontrol.keys())[0]}")
else:
    print("DİKKAT! Dosyalar arasında sütun farklılıkları var.")
    for i, (cols, file_list) in enumerate(sutun_kontrol.items()):
        print(f"\nGRUBU {i+1} ({len(file_list)} dosya):")
        print(f"Sütunlar: {cols}")
        print(f"Dosyalar: {file_list}")

✅ Dosyalar tarih sırasına (Ocak -> Aralık) göre dizildi.

📂 Bulunan Dosyalar (12 Adet):
['2024_November_TripData.csv', '2024_December_TripData.csv', '2025_January_TripData.csv', '2025_February_TripData.csv', '2025_March_TripData.csv', '2025_April_TripData.csv', '2025_May_TripData.csv', '2025_June_TripData.csv', '2025_July_TripData.csv', '2025_August_TripData.csv', '2025_September_TripData.csv', '2025_October_TripData.csv']
--------------------------------------------------

--- 🔍 SÜTUN KONTROL RAPORU ---
✅ MÜKEMMEL! Tüm dosyaların sütun isimleri ve sırası birebir aynı.
Sütunlar: ('ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual', 'duration_seconds')


In [5]:
import pandas as pd
import glob
import os
from datetime import datetime

# --- 1. DOSYALARI BUL VE SIRALA ---
# Dosyaları bul
dosyalar = glob.glob("*.csv")


def tarih_ayikla(dosya_ismi):
    try:
        
        parcalar = dosya_ismi.split('_')
        yil = parcalar[0]
        ay_ismi = parcalar[1]
        return datetime.strptime(f"{yil}-{ay_ismi}", "%Y-%B")
    except:
        return dosya_ismi 


dosyalar.sort(key=tarih_ayikla)

print(f"Toplam {len(dosyalar)} dosya kronolojik sırayla işlenecek.")
print(f"İlk dosya: {dosyalar[0]}")
print(f"Son dosya: {dosyalar[-1]}\n")

# --- SONUÇLARI BİRİKTİRECEĞİMİZ LİSTELER ---
list_flows = []       
list_hourly = []      
list_stations_out = [] 
list_stations_in = []  


cols_to_use = [
    'started_at', 
    'start_station_id', 'start_station_name', 'start_lat', 'start_lng',
    'end_station_id', 'end_lat', 'end_lng'
]

# --- ANA DÖNGÜ ---
for i, dosya in enumerate(dosyalar):
    print(f"⏳ [{i+1}/{len(dosyalar)}] Okunuyor ve Özetleniyor: {dosya} ...")
    
    # Dosyayı oku
    df = pd.read_csv(dosya, usecols=cols_to_use, low_memory=False)

    # Tarih işlemlerini yap
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['Year_Month'] = df['started_at'].dt.to_period('M').astype(str) 
    df['Day_of_Week'] = df['started_at'].dt.day_name()
    df['Hour'] = df['started_at'].dt.hour
    

    flow_agg = df.groupby([
        'Year_Month', 
        'start_station_id', 'end_station_id', 
        'start_lat', 'start_lng', 'end_lat', 'end_lng'
    ]).size().reset_index(name='Trip_Count')
    list_flows.append(flow_agg)
    
    # 2. HEATMAP İÇİN ZAMAN VERİSİ
    hourly_agg = df.groupby(['Day_of_Week', 'Hour']).size().reset_index(name='Trip_Count')
    list_hourly.append(hourly_agg)
    
    # 3. İSTASYON ANALİZİ (Giriş/Çıkış)
    # Çıkışlar
    out_agg = df.groupby(['start_station_id', 'start_station_name']).size().reset_index(name='Outflow')
    list_stations_out.append(out_agg)
    # Girişler
    in_agg = df.groupby(['end_station_id']).size().reset_index(name='Inflow')
    list_stations_in.append(in_agg)

print("Tüm dosyalar okundu. Parçalar birleştiriliyor")

# --- BİRLEŞTİRME VE KAYDETME ---

# DOSYA 1: monthly_flows.csv (Senin için - Harita)
final_flows = pd.concat(list_flows)
# Tekrar grupla (Dosyalar arası bölünmüş rotaları topla)
final_flows = final_flows.groupby([
    'Year_Month', 'start_station_id', 'end_station_id', 
    'start_lat', 'start_lng', 'end_lat', 'end_lng'
])['Trip_Count'].sum().reset_index()
final_flows.to_csv("monthly_flows.csv", index=False)
print(" 'monthly_flows.csv' oluşturuldu.")

# DOSYA 2: hourly_patterns.csv 
final_hourly = pd.concat(list_hourly)
final_hourly = final_hourly.groupby(['Day_of_Week', 'Hour'])['Trip_Count'].sum().reset_index()
final_hourly.to_csv("hourly_patterns.csv", index=False)
print("'hourly_patterns.csv' oluşturuldu.")

# DOSYA 3: station_stats.csv 
total_out = pd.concat(list_stations_out).groupby(['start_station_id', 'start_station_name'])['Outflow'].sum().reset_index()
total_in = pd.concat(list_stations_in).groupby(['end_station_id'])['Inflow'].sum().reset_index()

# Veriyi birleştir 
network_stats = pd.merge(total_out, total_in, left_on='start_station_id', right_on='end_station_id', how='outer')

# NaN (Boş) değerleri 0 yap
network_stats['Outflow'] = network_stats['Outflow'].fillna(0)
network_stats['Inflow'] = network_stats['Inflow'].fillna(0)

# İsimleri düzelt 
# Net Denge ve Toplam Hacim Hesapla
network_stats['Net_Balance'] = network_stats['Outflow'] - network_stats['Inflow'] # (+ Kaynak, - Kuyu)
network_stats['Total_Volume'] = network_stats['Outflow'] + network_stats['Inflow']

# Temizle ve Kaydet
cols = ['start_station_id', 'start_station_name', 'Outflow', 'Inflow', 'Net_Balance', 'Total_Volume']
network_stats = network_stats[cols]
network_stats.to_csv("station_stats.csv", index=False)
print(" 'station_stats.csv' oluşturuldu. (Bar Chart ve Analiz için)")

print(" TEBRİKLER! Veri Hazırlığı Tamamlandı.")

🚀 İŞLEM BAŞLIYOR... Toplam 12 dosya kronolojik sırayla işlenecek.
📂 İlk dosya: 2024_November_TripData.csv
📂 Son dosya: 2025_October_TripData.csv

⏳ [1/12] Okunuyor ve Özetleniyor: 2024_November_TripData.csv ...
⏳ [2/12] Okunuyor ve Özetleniyor: 2024_December_TripData.csv ...
⏳ [3/12] Okunuyor ve Özetleniyor: 2025_January_TripData.csv ...
⏳ [4/12] Okunuyor ve Özetleniyor: 2025_February_TripData.csv ...
⏳ [5/12] Okunuyor ve Özetleniyor: 2025_March_TripData.csv ...
⏳ [6/12] Okunuyor ve Özetleniyor: 2025_April_TripData.csv ...
⏳ [7/12] Okunuyor ve Özetleniyor: 2025_May_TripData.csv ...
⏳ [8/12] Okunuyor ve Özetleniyor: 2025_June_TripData.csv ...
⏳ [9/12] Okunuyor ve Özetleniyor: 2025_July_TripData.csv ...
⏳ [10/12] Okunuyor ve Özetleniyor: 2025_August_TripData.csv ...
⏳ [11/12] Okunuyor ve Özetleniyor: 2025_September_TripData.csv ...
⏳ [12/12] Okunuyor ve Özetleniyor: 2025_October_TripData.csv ...

🏗️ Tüm dosyalar okundu. Parçalar birleştiriliyor (Merge)...
✅ 'monthly_flows.csv' oluşturuld